In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [9,279 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [89.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-

In [2]:
import requests
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

In [3]:
# for selenuim
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(chrome_options=options) 
wait = WebDriverWait(driver,10)

# netkeiba.comの検索ページを開く
driver.get('https://db.netkeiba.com/?pid=race_search_detail')
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

In [ ]:
# 検索条件入力

# 種別 : 芝 + ダート

# 芝
driver.find_element_by_id('check_track_1').click()
# ダート
driver.find_element_by_id('check_track_2').click()

# 期間 : 2015/1 ~ 現在
year = datetime.now().year
month = datetime.now().month

# 期間：開始年
start_year_element = driver.find_element_by_name('start_year')
start_year_select = Select(start_year_element)
start_year_select.select_by_value('2015')
# 期間：開始月
start_mon_element = driver.find_element_by_name('start_mon')
start_mon_select = Select(start_mon_element)
start_mon_select.select_by_value('1')
# 期間：終了年
end_year_element = driver.find_element_by_name('end_year')
end_year_select = Select(end_year_element)
end_year_select.select_by_value(str(year))
# 期間：終了月
end_mon_element = driver.find_element_by_name('end_mon')
end_mon_select = Select(end_mon_element)
end_mon_select.select_by_value(str(month))

# 競馬場：すべて
driver.find_element_by_id('check_Jyo_01').click()
driver.find_element_by_id('check_Jyo_02').click()
driver.find_element_by_id('check_Jyo_03').click()
driver.find_element_by_id('check_Jyo_04').click()
driver.find_element_by_id('check_Jyo_05').click()
driver.find_element_by_id('check_Jyo_06').click()
driver.find_element_by_id('check_Jyo_07').click()
driver.find_element_by_id('check_Jyo_08').click()
driver.find_element_by_id('check_Jyo_09').click()
driver.find_element_by_id('check_Jyo_10').click()

# 表示件数を選択(20,50,100の中から最大の100へ)
list_element = driver.find_element_by_name('list')
list_select = Select(list_element)
list_select.select_by_value("100")

# フォームを送信
frm = driver.find_element_by_css_selector("#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

In [ ]:
%%time

with open('race_url.txt', 'w+') as f:
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        all_rows = driver.find_element_by_class_name('race_table_01').find_elements_by_tag_name("tr")
        for row in range(1, len(all_rows)):
            r = all_rows[row].find_elements_by_tag_name("td")[3].text
            if r in ["10", "11"]:
                race_href=all_rows[row].find_elements_by_tag_name("td")[4].find_element_by_tag_name("a").get_attribute("href")
                f.write(race_href+"\n")
        try:
            target = driver.find_elements_by_link_text("次")[0]
            driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
        except IndexError:
            break

In [6]:
urls = []

with open('race_url.txt', 'r') as f:
    for line in f.readlines():
        urls.append(line.replace('\n', ''))

print(len(urls))

3187 601


In [7]:
def convert_num(value:str, flg:bool):
    '''
    str型を数値int/float型に変更
    null、空文字、変更不可の場合Nanを返却
    '''
    
    if value is None or value == '':
        return np.nan
    
    try:
        if flg:
            ret = int(value)
        else:
            ret = float(value)
        return ret

    except ValueError:
        return np.nan

In [23]:
def get_race(url, text):
    '''
    レース情報取得
    '''
    try:
        # レースID
        race_id = int(url.split('/')[-2])

        bs = BeautifulSoup(text)
        title = bs.find('div', class_='data_intro')
        # 競馬場
        # 2020年5月16日 2回東京7日目 4歳以上オープン  (国際)(指)(別定)
        p_title = title.find('p', class_='smalltxt').get_text()
        p_title_list = p_title.split(' ')
        place = re.sub('[0-9]+|回|日目', '', p_title_list[1])
        # 芝/ゲート, 距離, 天候, 芝状態, 日時
        span_title = title.find('span').get_text()
        span_title_list = span_title.split('\xa0/\xa0')
        track = span_title_list[0][0]
        direction = span_title_list[0][1]
        length = convert_num(re.search('\d+', span_title_list[0]).group(), True)
        weather = span_title_list[1].split(' : ')[1]
        condition = span_title_list[2].split(' : ')[1]
        date_ = datetime.strptime(p_title_list[0], '%Y年%m月%d日')
            
    except Exception as e:
        print(f'{url} : {e}')
        # return []
    
    return [race_id, date_, place, track, length, direction, weather, condition]

def get_horse(race_id, text):
    try:
        bs = BeautifulSoup(text)
        table = bs.find('table', class_='race_table_01 nk_tb_common')

        title = []
        for th in table.find_all('tr')[0].find_all('th'):
            title.append(th.get_text())
        
        trs = table.find_all('tr')[1:]
        data = []
        for i in range(len(trs)):
            tds = trs[i].find_all('td')
            temp_data = []
            for item in ['着順', '馬番', '馬名', '騎手', '人気']:
                td = tds[title.index(item)]
                if item in ['馬名', '騎手']:
                    temp_data.append(td.find('a').get('href').split('/')[2])
                else:
                    temp_data.append(td.get_text())
            temp_data.append(None)

            data.append(temp_data)

    except Exception as e:
        print(e)

    return data

In [28]:
main_time = time.time()


race_data = []

for i, url in enumerate(urls):
    start_time = time.time()
    # print(url)
    time.sleep(5)

    res = requests.get(url)
    res.encoding = res.apparent_encoding

    race_info = get_race(url, res.content)
    horse_info = get_horse(race_info[0], res.content)

    for horse in horse_info:
        race_data.append(race_info+horse)
    
    print(f'{i+1}/{len(urls)} : {(time.time() - start_time) / 60} min. Total: {(time.time() - main_time) / 60} min.')

df = pd.DataFrame(race_data, columns=['race_id', 'date', 'place', 'track', 'length', 'direction', 'weather', 'condition', 'rank', 'horse_num', 'horse_id', 'jockey_id', 'popularity', 'weight'])
df.to_csv('race_data.csv', index=False, header=True)

1/3187 : 0.10935970147450765 min. Total: 0.10936211347579956 min.
